# SI 618 Homework 1
---------------------------
Name: Junqi Chen 

Uniqname: junqich

---------------------------

In [183]:
import numpy as np
import pandas as pd

## Introduction

In [184]:
# Question 1
df = pd.read_csv("book_data.csv")
df.head()

,book_authors,book_desc,book_edition,book_format,book_isbn,book_pages,book_rating,book_rating_count,book_review_count,book_title,genres,image_url
0,Kristin Hannah,"Alaska, 1974.Unpredictable. Unforgiving. Untam...",NaN,Kindle Edition,NaN,435 pages,4.33,146505,17438,The Great Alone,Fiction|Historical|Historical Fiction|Audioboo...,https://images.gr-assets.com/books/1501852423l...
1,Madeline Miller,"In the house of Helios, god of the sun and mig...",NaN,Hardcover,9.780000e+12,393 pages,4.34,61357,9369,Circe,Fantasy|Fiction|Fantasy|Mythology|Historical|H...,https://images.gr-assets.com/books/1508879575l...
2,Holly Black,Of course I want to be like them. They’re beau...,NaN,Hardcover,9.780000e+12,370 pages,4.18,62897,11948,The Cruel Prince,Fantasy|Young Adult|Fairies|Fae,https://images.gr-assets.com/books/1493047153l...
3,A.J. Finn,Anna Fox lives alone—a recluse in her New York...,NaN,Kindle Edition,NaN,449 pages,3.97,175678,20645,The Woman in the Window,Mystery|Thriller|Fiction|Suspense|Thriller|Mys...,https://images.gr-assets.com/books/1528225499l...
4,Tara Westover,An unforgettable memoir in the tradition of Th...,NaN,Hardcover,9.780000e+12,334 pages,4.47,128337,16424,Educated,Nonfiction|Autobiography|Memoir|Biography,https://images.gr-assets.com/books/1506026635l...


In [185]:
# Question 2
df_clean = df.drop_duplicates(subset=["book_title"]) # remove duplicate according to `book_title`
print("There are", len(df_clean), "unique books.")

There are 1125 unique books.


In [186]:
# Question 3
two_authors_list = [aus for aus in df_clean.book_authors if aus.count("|") == 1]
print("There are", len(two_authors_list), "books have two authors.")

There are 107 books have two authors.


## Length

I am going to drop some columns because there are no record on page number (NaN) for some of the books. Then we can successfully add a integer column of page number to the dataframe.

In [187]:
# Question 4
df_clean = df_clean.dropna(subset=["book_pages"])
print("There are", len(df_clean), "unique books left after dropping NaN in `book_pages`.")
df_clean["book_pages_int"] = [int(str.split()[0]) for str in df_clean.book_pages]
df_clean.book_pages_int.head(10)

There are 1056 unique books left after dropping NaN in `book_pages`.


0    435
1    393
2    370
3    449
4    334
5    525
6    308
7    352
8    346
9    504
Name: book_pages_int, dtype: int64

In [188]:
# Question 5
pages_median = df_clean.book_pages_int.median()
print("The median number of pages is", pages_median)

The median number of pages is 326.5


In [189]:
# Question 6
print("The minimum number of pages is", df_clean.book_pages_int.min())
print("The maximum number of pages is", df_clean.book_pages_int.max())

The minimum number of pages is 0
The maximum number of pages is 992


In [190]:
# Question 7
def countAvgPage(authorNum = 1, df_clean = df_clean):
    n_author_index = [i for i in df_clean.index if df_clean.book_authors[i].count("|") == authorNum - 1]
    df_n_author = df_clean.loc[n_author_index]
    return df_n_author.book_pages_int.mean()

for n in range(1, 4):
    print("The average number of pages for books written by", n, "author(s) is", countAvgPage(n))

The average number of pages for books written by 1 author(s) is 333.3510292524377
The average number of pages for books written by 2 author(s) is 274.77777777777777
The average number of pages for books written by 3 author(s) is 277.75


So we can find out that the average number of pages for books written by 1 author is longer than 2 authors, indicating that **having more than 1 author *does not* result in a longer book on average**. The following is an alternative way to solve this question (I put it here for reference).

In [191]:
# Alternative way for Question 7
def countAvgPage_Alt(authorNum = 1):
    n = 0
    count = 0
    for i in df_clean.index:
        if df_clean.book_authors[i].count("|") == authorNum - 1:
            n += 1
            count += df_clean.book_pages_int[i]
    return count / n

for n in range(1, 4):
    print("The average number of pages for books written by", n, "author(s) is", countAvgPage_Alt(n))

The average number of pages for books written by 1 author(s) is 333.3510292524377
The average number of pages for books written by 2 author(s) is 274.77777777777777
The average number of pages for books written by 3 author(s) is 277.75


## Rating

In [192]:
# Question 8
df_clean = df_clean.dropna(subset=["book_rating"])
print("There are", len(df_clean[df_clean.book_rating >= 4.5]), "books have at least a rating of 4.5")

There are 57 books have at least a rating of 4.5


In [193]:
# Question 9
print("There are", len(df_clean[df_clean.book_rating <= 4]), "books have at most a rating of 4")

There are 473 books have at most a rating of 4


In [194]:
# Question 10
df_clean["book_rating_discret"] = df_clean["book_rating"].apply(np.floor)
df_clean["book_rating_discret"].head()

0    4.0
1    4.0
2    4.0
3    3.0
4    4.0
Name: book_rating_discret, dtype: float64

In [195]:
# Question 11
df_clean["book_authors_num"] = [(aus.count("|") + 1) for aus in df_clean["book_authors"]]
df_groupby_rating = df_clean.groupby(by = ["book_rating_discret"]).mean()

avg_author_num = df_groupby_rating["book_authors_num"]
for i in avg_author_num.index:
    print("For discretized ratings", i, ", the average number of authors is", avg_author_num[i])

For discretized ratings 2.0 , the average number of authors is 1.3333333333333333
For discretized ratings 3.0 , the average number of authors is 1.223194748358862
For discretized ratings 4.0 , the average number of authors is 1.3356047700170357
For discretized ratings 5.0 , the average number of authors is 1.0


In [196]:
# Question 12
avg_review_num = df_groupby_rating["book_review_count"]
for i in avg_review_num.index:
    print("For discretized ratings", i, ", the average number of reviews is", avg_review_num[i])

For discretized ratings 2.0 , the average number of reviews is 251.33333333333334
For discretized ratings 3.0 , the average number of reviews is 896.9321663019693
For discretized ratings 4.0 , the average number of reviews is 916.2879045996593
For discretized ratings 5.0 , the average number of reviews is 0.7777777777777778


## Genres

In [197]:
# Question 13
df_onehot = df_clean["genres"].str.get_dummies() # we apply one-hot encoding to column `genres`
df_new = pd.concat([df_clean, df_onehot], axis=1) # cancat the onehot df with original df to make a new df
df_new.head()

,book_authors,book_desc,book_edition,book_format,book_isbn,book_pages,book_rating,book_rating_count,book_review_count,book_title,...,Womens,Womens Fiction,World War I,World War II,Writing,Young Adult,Young Adult Contemporary,Young Adult Fantasy,Young Adult Historical Fiction,Young Adult Romance
0,Kristin Hannah,"Alaska, 1974.Unpredictable. Unforgiving. Untam...",NaN,Kindle Edition,NaN,435 pages,4.33,146505,17438,The Great Alone,...,0,0,0,0,0,0,0,0,0,0
1,Madeline Miller,"In the house of Helios, god of the sun and mig...",NaN,Hardcover,9.780000e+12,393 pages,4.34,61357,9369,Circe,...,0,0,0,0,0,0,0,0,0,0
2,Holly Black,Of course I want to be like them. They’re beau...,NaN,Hardcover,9.780000e+12,370 pages,4.18,62897,11948,The Cruel Prince,...,0,0,0,0,0,1,0,0,0,0
3,A.J. Finn,Anna Fox lives alone—a recluse in her New York...,NaN,Kindle Edition,NaN,449 pages,3.97,175678,20645,The Woman in the Window,...,0,0,0,0,0,0,0,0,0,0
4,Tara Westover,An unforgettable memoir in the tradition of Th...,NaN,Hardcover,9.780000e+12,334 pages,4.47,128337,16424,Educated,...,0,0,0,0,0,0,0,0,0,0


In [198]:
# Question 14
genres_list = df_onehot.columns

avg_ratings = [df_new.groupby(genre).mean()["book_rating"][1] for genre in genres_list]
median_ratings = [df_new.groupby(genre).median()["book_rating"][1] for genre in genres_list]
df_genre_rating =pd.DataFrame({"avg_rating": avg_ratings,
                               "median_rating": median_ratings},
                              index = genres_list)
df_genre_rating

,avg_rating,median_rating
16th Century,4.130000,4.130
20th Century,4.000000,4.000
Abandoned,3.557692,3.630
Abuse,4.395000,4.300
Academic,4.190000,4.110
...,...,...
Young Adult,3.942812,3.940
Young Adult Contemporary,3.770000,3.785
Young Adult Fantasy,3.937273,3.940
Young Adult Historical Fiction,3.900000,3.900


In [199]:
# Question 15
avg_authorNum = [df_new.groupby(genre).mean()["book_authors_num"][1] for genre in genres_list]
median_authorNum = [df_new.groupby(genre).median()["book_authors_num"][1] for genre in genres_list]
df_genre_authorNum =pd.DataFrame({"avg_authorNum": avg_authorNum,
                                  "median_authorNum": median_authorNum},
                                 index = genres_list)
df_genre_authorNum

,avg_authorNum,median_authorNum
16th Century,1.00000,1.0
20th Century,1.00000,1.0
Abandoned,1.00000,1.0
Abuse,1.75000,2.0
Academic,1.00000,1.0
...,...,...
Young Adult,1.46875,1.0
Young Adult Contemporary,1.12500,1.0
Young Adult Fantasy,1.00000,1.0
Young Adult Historical Fiction,1.00000,1.0
